In [1]:
pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-1c0m7qei/kobert-tokenizer_93f0ca5a6ff246de9e09974ce5378414
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-1c0m7qei/kobert-tokenizer_93f0ca5a6ff246de9e09974ce5378414
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=5378f36efbd0d010882d8d76606a242d5d2afbc05f2cbb038e47ccde01925442
  Stored in directory: /tmp/pip-ephem-wheel-cache-0tp41hci/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [5]:
pip install SentencePiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.1 MB/s eta 0:00:00


In [1]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1', sp_model_kwargs={'nbest_size': -1, 'alpha': 0.6, 'enable_sampling': True})
tokenizer.encode("한국어 모델을 공유합니다.")
[2, 4958, 6855, 2046, 7088, 1023, 7063, 7843, 54, 3]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


[2, 4958, 6855, 2046, 7088, 1023, 7063, 7843, 54, 3]

In [3]:
import torch
from transformers import BertModel
model = BertModel.from_pretrained('skt/kobert-base-v1')
text = "한국어 모델을 공유합니다."
inputs = tokenizer.batch_encode_plus([text])
out = model(input_ids = torch.tensor(inputs['input_ids']),
             attention_mask = torch.tensor(inputs['attention_mask']))
out.pooler_output.shape
torch.Size([1, 768])

config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

torch.Size([1, 768])

In [4]:
text = '“실제 역사를 거의 그대로 다루다보니 보면서 화가 나긴 하지만 영화적으로 흥미롭게 구성을 잘해서 몰입도가 높아요. 몇 번을 봐도 볼 때마다 새롭게 보이는 면이 있구요. 의미와 재미를 다 잡은 작품이에요'

In [5]:
import torch
import torch.nn as nn
from transformers import BertModel

# Define a simple classifier on top of KoBERT
class SentimentClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super().__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(p=0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        # Extract the last hidden state of the token `[CLS]` for classification task
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.pooler_output  # shape: (batch_size, hidden_size)
        cls_output = self.dropout(cls_output)
        # We use the output of the `[CLS]` token for our classification
        logits = self.classifier(cls_output)
        return logits

# Instantiate the model
# For binary sentiment analysis, num_classes will be 2 (positive, negative)
model = SentimentClassifier('skt/kobert-base-v1', num_classes=2)

# Tokenize the input text
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

inputs = tokenizer.encode_plus(
    "실제 역사를 거의 그대로 다루다보니 보면서 화가 나긴 하지만 영화적으로 흥미롭게 구성을 잘해서 몰입도가 높아요. 몇 번을 봐도 볼 때마다 새롭게 보이는 면이 있구요. 의미와 재미를 다 잡은 작품이에요",
    return_tensors='pt',
    max_length=512,
    truncation=True,
    padding='max_length',
    add_special_tokens=True
)

# Make predictions with the model
with torch.no_grad():
    logits = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])

# Apply softmax to logits to get probabilities
probabilities = nn.functional.softmax(logits, dim=1)
print(probabilities)

# Assuming class 0 is 'negative' and class 1 is 'positive', we take the argmax to get the most likely class
predicted_class = torch.argmax(probabilities, dim=1)
print(predicted_class)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


tensor([[0.4982, 0.5018]])
tensor([1])


In [6]:
# 결과
# tensor([[0.4982, 0.5018]]) # 거의 반반 확률로 neg / pos 의견
# tensor([1]) # 결론은 positive로